### 读取数据集，并且转变prompt 为 labels

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"  # 指定使用 GPU 3

In [2]:
import os
import copy

import torch
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, EvalPrediction
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay

2024-06-30 01:04:55.741598: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-30 01:04:55.803903: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 01:04:55.803947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 01:04:55.805436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-30 01:04:55.814763: I tensorflow/core/platform/cpu_feature_guar

In [3]:
PATH = "/home/mcq/GitHub/aes2/kaggle/input/learning-agency-lab-automated-essay-scoring-2/"


train_df = pd.read_csv(PATH + "train.csv")
#data1 = pd.read_csv(PATHS.train_path)
persuade = pd.read_csv('/home/mcq/GitHub/aes2/train_data/persuade_2.0.csv')

In [35]:
## 将prompt转变为数字
persuade["label"] =  persuade["prompt_name"].astype("category").cat.codes

In [36]:
##计算交叉点
intersection = pd.merge(train_df, persuade, on="full_text", how="inner")[["essay_id", "full_text", "score", "prompt_name","label"]].reset_index(drop=True)
persuade.rename(columns={'essay_id_comp': 'essay_id'}, inplace=True)
persuade.rename(columns={'holistic_essay_score': 'score'}, inplace=True)

### 数据集去除交叉数据， 返回大数据集labels

In [38]:
difference = train_df[~train_df["essay_id"].isin(intersection["essay_id"])].reset_index(drop=True)



###直接替换SKF的train就可以 
difference1 = persuade[~persuade["essay_id"].isin(intersection["essay_id"])].reset_index(drop=True)
difference1 = difference1.iloc[:, :3] 
skf_train = pd.concat([train_df, difference1], axis=0, ignore_index=True) 
### skf_train就是加过额外数据集的train了， 



difference2 = persuade[~persuade["essay_id"].isin(intersection["essay_id"])].reset_index(drop=True)
# print("len(intersection):", len(difference))
# print("len(difference):", len(difference2))
### 有25996个数据集
###首先drop掉0-6之间的topic数据点

len(intersection): 4436
len(difference): 25996


In [47]:
mask = difference2['label'].isin([1,  5 , 0 , 7 , 6 , 8 ,14])
df_dropped = difference2[~mask]
GFX_add7 = difference2[~difference2["essay_id"].isin(df_dropped["essay_id"])].reset_index(drop=True)
##GFX_add7长度为12873，是在7个topic之内的点

In [66]:
GKF_train = df_dropped ###长度为13121,是topic之外的点
GKF_train = GKF_train.iloc[:, :3]  ##数据规范化处理

,essay_id,full_text,score
0,423A1CA112E2,Phones\n\nModern humans today are always on th...,3
1,BC75783F96E3,This essay will explain if drivers should or s...,4
2,74C8BC7417DE,Driving while the use of cellular devices\n\nT...,2
3,A8445CABFECE,Phones & Driving\n\nDrivers should not be able...,3
4,6B4F7A0165B9,Cell Phone Operation While Driving\n\nThe abil...,4


### 在原始数据集中添加额外的7topics之内的代码，对应的是GFX_add7

In [56]:
import topicfeaturesmall
PATH = "/home/mcq2/GitHub/aes2/kaggle/input/learning-agency-lab-automated-essay-scoring-2/"
train = pd.read_csv(PATH + "train.csv")
topic = topicfeaturesmall.predict_chunk(train)
groups_1= topic['topic'] ## 读取一万七的topic

Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

/home/mcq2/anaconda3/envs/aes/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [58]:
groups_2 = GFX_add7['label'] ##读取额外的topic
groups = pd.concat([groups_1, groups_2], axis=0, ignore_index=True)

In [ ]:
GFX_add7 = GFX_add7.iloc[:, :3] ### 注意这个一定要放在group提取之后，因为这里已经把topic列删掉了

In [61]:
train_2 = pd.concat([train, GFX_add7], axis=0, ignore_index=True) ###把额外的7个topic之内的代码整合进原有的数据集，这里的代码结构和train一样

### 上面的train2 用来代替train跑feature和feature_selection的代码！！

### 下面的代码就是分组，一个在GKF里面，一个在GKF外面，放在baseline代码的GKF就可以了！

In [62]:
#X = train_feats[feature_names].astype(np.float32).values 

X = train_2 # ### 在这里的时候使用train_2跑feature_selection之后的pickel文件
y_split = train_2['score'].astype(int).values

In [64]:
###GKF代码, 只加入了一万二的topic=7的额外数据集
###这个代码可以与下面的进行整合
### 写在这里是为了让你参考它写SKF的代码的，
### 所以我注释掉了
# from sklearn.model_selection import GroupKFold
# LOAD = True 

# n_splits = 7 ## 如果变更为大数据集，这个_split要更改
# models = []


# if not LOAD:
#     for i in range(n_splits):
#         models.append(lgb.Booster(model_file=f'kaggle/input/aes-lgbm/fold_{i+1}.txt'))
# else:
#     group_kfold = GroupKFold(n_splits=n_splits)
#     f1_scores = []
#     kappa_scores = []
#     models = []
#     predictions = []
 
#     for i, (train_index, test_index) in enumerate(group_kfold.split(X, y_split, groups)):
#         X_train_fold, X_test_fold = X[train_index], X[test_index]
#         y_train_fold, y_test_fold, y_test_fold_int = y_split[train_index], y_split[test_index], y_split[test_index]
#         # 可以通过这三行代码看一下数据，直接删掉就可以
#         # print(f"Fold {i}:")
#         # print(f"  Train: index={train_index}, group={groups[train_index]}")
#         # print(f"  Test:  index={test_index}, group={groups[test_index]}")

### 在for循环里加入额外的训练数据集

In [78]:
X_2 = GKF_train ##注意这里的X2要更改为GKF_train 的feature select 之后的文件
#X_2 = train_feats[feature_names].astype(np.float32).values 
y_split_2 = GKF_train['score'].astype(int).values

In [79]:
skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=0)
train_index_2 = []
i=1
for train_index, test_index in skf.split(X_2,y_split_2):
    train_index_2.append(train_index)

In [82]:
from sklearn.model_selection import GroupKFold
LOAD = True 

n_splits = 7 ## 如果变更为大数据集，这个_split要更改
models = []


if not LOAD:
    for i in range(n_splits):
        models.append(lgb.Booster(model_file=f'kaggle/input/aes-lgbm/fold_{i+1}.txt'))
else:
    group_kfold = GroupKFold(n_splits=n_splits)
    f1_scores = []
    kappa_scores = []
    models = []
    predictions = []
 
    for i, (train_index, test_index) in enumerate(group_kfold.split(X, y_split, groups)):
        ### X, y_split是已经加入额外数据集的代码
        ###你做SKF也只要之前修改数据集就完事了
        X_train_fold_1, X_test_fold = X[train_index], X[test_index]
        X_train_fold_2 = X_2[train_index_2[i]]
        X_train_fold = y_train_fold = np.hstack((X_train_fold_1, X_train_fold_2))
        ## 链接2个训练集

        y_train_fold_1, y_test_fold, y_test_fold_int = y_split[train_index], y_split[test_index], y_split[test_index]
        y_train_fold_2 = y_split_2[train_index_2[i]]
        y_train_fold = np.hstack((y_train_fold_1, y_train_fold_2))
        # 可以通过这三行代码看一下数据，直接删掉就可以
        # print(f"Fold {i}:")
        # print(f"  Train: index={train_index}, group={groups[train_index]}")
        # print(f"  Test:  index={test_index}, group={groups[test_index]}")

In [4]:
# 统计单词数量
train_df['word_count_pure'] = train_df['full_text'].apply(lambda x: len(x.split()))
persuade['word_count_pure'] = train_df['full_text'].apply(lambda x: len(x.split()))
